# Introduction

* Thoughts on improve the profitability
> Should find pairs with stronger relationship
>
> Should use some other econometrics model to calculate the spread (?Can we use something other than OLS?)

This notebook aims for providing an experiment of traditional pair trading with parameter autotuning grid search.

This experiment is based on [backtrader](https://www.backtrader.com/) which has an agnosticism philosophy that the data come in as a per-sample basis. Indicators are recalculate when new data come in.

In [6]:
import warnings
warnings.filterwarnings('ignore')

import os
import pyfolio
import json
import numpy as np
import pandas as pd
import statsmodels.api as sm
import backtrader as bt

from tqdm import tqdm
from itertools import combinations
from utils.gridsearch import gridsearch
from utils.read2df import read2df
from utils.cointncorr import CointnCorr
from utils.resanalyse import res_analyse
from utils.pyfolioprocess import pyfolio_process
from envs.env_gridsearch import KellyCriterionIndicator, PairTrading

os.makedirs("result/gridsearch", exist_ok=True)

cointncorrtxt = f"result/gridsearch/cointncorr.txt"
os.remove(cointncorrtxt) if os.path.exists(cointncorrtxt) else None

for root, dirs, files in os.walk(f"result/gridsearch/"):
    for file in files:
        os.remove(os.path.join(root, file))

# Data Preparation

Download historical data for `symbols` after `start_date` with selected `freqs` from [`binance-public-data`](https://github.com/binance/binance-public-data/tree/master/python)

We will train data from `start_date` until `trade_date`, and start trade after `trade_date`.

In [7]:
# symbols = ['BTCUSDT', 'ETHUSDT', 'LTCUSDT', 'XMRUSDT', 'BNBUSDT', 'ADAUSDT', 'DOGEUSDT', 'SOLUSDT', 'TRXUSDT']
symbols = ['BTCUSDT', 'BTCUSD', 'BTCTUSD', 'BTCUSDC', 'BTCBUSD', 'BTCDAI']
# symbols = ['USDCUSDT', 'DAIUSDT', 'TUSDUSDT', 'BUSDUSDT', 'USDCTUSD', 'USDCBUSD', 'DAIBUSD', 'TUSDBUSD', 'BUSDDAI']
start_date = '2022-01-01'
trade_date = '2023-01-01'

# freqs = {'1h':60, '2h':120, '4h':240, '6h':360, '8h':480, '12h':720, '1d':1440}
freqs = {'3m':3, '5m':5, '15m':15, '30m':30}

In [8]:
%%capture
if symbols is None:
    !python binance-public-data/python/download-kline.py \
        -i {" ".join(list(freqs.keys()))} -startDate {start_date} -t spot -skip-daily 1
else:
    !python binance-public-data/python/download-kline.py \
        -s {" ".join(symbols)} -i {" ".join(list(freqs.keys()))} -startDate {start_date} -t spot -skip-daily 1

In [9]:
'''
Read the downloaded OHLCV data into `pandas` dataframe
'''

# dfs = read2df(symbols, freqs)
dfs = read2df(symbols, freqs)

# have a preview
dfs[0]

,time,open,high,low,close,volume,tic,itvl,datetime
0,1640995379999,46215.75,46382.65,46212.42,46373.28,37.06541,BTCBUSD,3m,2022-01-01 00:02:59.999
1,1640995379999,46329.94,46401.68,46329.94,46382.12,0.31340,BTCDAI,3m,2022-01-01 00:02:59.999
2,1640995379999,46245.70,46324.83,46243.70,46324.83,0.12080,BTCTUSD,3m,2022-01-01 00:02:59.999
3,1640995379999,46216.40,46375.85,46216.40,46372.24,1.66527,BTCUSDC,3m,2022-01-01 00:02:59.999
4,1640995379999,46216.93,46381.69,46208.37,46368.73,134.25635,BTCUSDT,3m,2022-01-01 00:02:59.999
...,...,...,...,...,...,...,...,...,...
1211510,1698796799999,34648.10,34664.70,34641.51,34643.94,0.40204,BTCBUSD,3m,2023-10-31 23:59:59.999
1211511,1698796799999,34667.04,34675.74,34643.61,34643.61,0.03416,BTCDAI,3m,2023-10-31 23:59:59.999
1211512,1698796799999,34645.96,34662.58,34645.96,34648.08,3.79823,BTCTUSD,3m,2023-10-31 23:59:59.999
1211513,1698796799999,34663.44,34676.41,34656.42,34656.42,0.22968,BTCUSDC,3m,2023-10-31 23:59:59.999


In [10]:
'''
Set data before `trade_data` as training data, after `trade_data` is trade_data
'''

trains, tests = [], []
for i in range(len(dfs)):
    trains.append(dfs[i][(dfs[i]['datetime'] > start_date) & (dfs[i]['datetime'] < trade_date)].reset_index(drop=True))
    tests.append(dfs[i][dfs[i]['datetime'] >= trade_date].reset_index(drop=True))

# Cointegration and Correlation

We need to make sure that our data is capable for pair trading. 
At least it should have the tendency to merge together therefore we test the coint and corr between every possible two pairs 

Calculate daily coint and corr for all the pairs
Consider 1 day with 1440 minutes.

Delete the pickle file in the `\result` if want to recalculate

In [11]:
'''
Load data from pickle or recalculate
'''

import pickle

# If there is already a pickle file
pickle_file = 'result/cointncorr.pickle'

if os.path.exists(pickle_file):
    with open('result/cointncorr.pickle', 'rb') as pk:
        data = pickle.load(pk)

    freq_position = list(freqs.keys()).index(data[1])

    df0 = dfs[freq_position][dfs[freq_position]['tic']==data[0][0]].reset_index(drop=True)
    df1 = dfs[freq_position][dfs[freq_position]['tic']==data[0][1]].reset_index(drop=True)

    pair, best_freq = data 
    best_pair = f"{pair[0]}_{pair[1]}"

    print("===========================================")
    print(f"Data loaded as: {best_pair} under {best_freq} interval")
    print("===========================================")

else: 
    # If there is no pickle file, then recalculate
    # Takes a looooong time
    tables = CointnCorr(trains, freqs).tabulate()

    with open(cointncorrtxt, "a") as f:
        for k, v in tables.items():
            f.write(f"{k}\n")
            f.write(f"{v}\n\n")
    f.close()

    best_value = 0
    for key in tables.keys():
        for freq in freqs:
            rel = tables[key].at['coint', freq] + tables[key].at['corr', freq]
            if rel > best_value:
                best_value = rel
                best_pair = key
                best_freq = freq

    print("===========================================")
    print(f"Best trading pairs shall be: {best_pair} under {best_freq} interval")
    print(f"the coint is {round(tables[best_pair].at['coint', best_freq]*100, 2)}%")
    print(f"and the corr is {round(tables[best_pair].at['corr', best_freq],3)}")
    print("===========================================")

Data loaded as: BTCBUSD_BTCTUSD under 3m interval


In [16]:
cointncorr = best_pair.split("_"), best_freq

with open('result/cointncorr.pickle', 'wb') as pk:
    pickle.dump(cointncorr, pk)
    
freq_pos = list(freqs.keys()).index(best_freq)

traindata0 = trains[freq_pos][trains[freq_pos]['tic']==best_pair.split('_')[0]].reset_index(drop=True)
traindata1 = trains[freq_pos][trains[freq_pos]['tic']==best_pair.split('_')[1]].reset_index(drop=True)

# How come a datetime of 2022(ish) is converted to 738786.635416655???
datafeed0 = bt.feeds.PandasData(
        dataname=traindata0,
        datetime='datetime',
        open='open',
        high='high',
        low='low',
        close='close',
        volume='volume',
        openinterest=None,
    )

datafeed1 = bt.feeds.PandasData(
        dataname=traindata1,
        datetime='datetime',
        open='open',
        high='high',
        low='low',
        close='close',
        volume='volume',
        openinterest=None,
    )

datafeeds = [datafeed0, datafeed1]
print(f"length of train set is {len(traindata0)}")

length of train set is 130140


# Execute the Strategy

We should firstly have a test run on the strategy to make sure it works.

Submitted a PR for Backtrader-OLS results

> https://github.com/mementum/backtrader/pull/487

In [17]:
def cerebro_run(datafeeds, param):
    # Create a Cerebro instance and add the data feed
    cerebro = bt.Cerebro()
    # TODO: should I include `best_pair` as a parameter?
    cerebro.adddata(datafeeds[0], name=best_pair.split('_')[0])
    cerebro.adddata(datafeeds[1], name=best_pair.split('_')[1])

    # Set up other parameters for backtest
    cerebro.broker.set_cash(100000)  # Set initial capital
    # cerebro.broker.setcommission(commission=0.001) 

    # comminfo = PairTradingCommInfo(commission=0.002, margin=1000, mult=10)
    # cerebro.broker.addcommissioninfo(comminfo)

    cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='timereturns', compression=60)
    cerebro.addanalyzer(bt.analyzers.Returns, _name='Returns')
    cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='tradeanalyzer')
    # cerebro.addsizer(KellyCriterionSizer)

    cerebro.addstrategy(PairTrading, **param)
    strats = cerebro.run()
    return strats

In [18]:
# A test run
# https://github.com/mementum/backtrader/blob/master/backtrader/indicators/ols.py
# It always returns weird error like the number of params

# 1. The default OLS indicator is precarious, always weird errors like num of params, or sometimes index error
# 2. The custom indicator is precious as well. not trustworthy

param = {'OPEN_THRE':2, 'CLOS_THRE':0.1, 'period':1000, 'verbose':2, 'prefix':'Experiment'}
experiment = cerebro_run(datafeeds, param)

ValueError: not enough values to unpack (expected 2, got 1)

# Grid Search the Strategy

The grid search on all the possible results for an optimal outcome

Define `scoring` function and `param_grid` for grid search

then `gridsearch` from `utils.gridsearch`

In [ ]:
datafeeds = [datafeed0, datafeed1]
freq_adjust = freqs[best_freq]

'''
Seems longer period provides better result
I guess its because the zscore is relatively stabler
'''
param_grid = {
    'OPEN_THRE': [2.2], # np.arange(2.2, 2.3, 0.1), 
    'CLOS_THRE': [0.1], # np.arange(0.1, 0.1, 0.1), 
    'period': np.arange(int(1300/freq_adjust), int(1400/freq_adjust), int(100/freq_adjust)),
    'verbose': [0],
    'prefix': ['Gridsearch']
}

def scoring(strats):
    score = strats[0].analyzers.Returns.get_analysis()['rtot']
    return score

# included tqdm for progress bar
# Take a loooong time
best_profit, best_params, best_result = gridsearch(cerebro_run, param_grid, scoring, datafeeds)

print(f"Best parameters as: {best_params}")
print(f"Best profit as: {best_profit}")

# Trade with test data
Use the parameter tra

In [ ]:
freq_position = list(freqs.keys()).index(best_freq)

testdata0 = tests[freq_position][tests[freq_position]['tic']==best_pair.split('_')[0]].reset_index(drop=True)
testdata1 = tests[freq_position][tests[freq_position]['tic']==best_pair.split('_')[1]].reset_index(drop=True)

test_datafeed0 = bt.feeds.PandasData(
        dataname=testdata0,
        datetime='datetime',
        open='open',
        high='high',
        low='low',
        close='close',
        volume='volume',
        openinterest=None,
    )

test_datafeed1 = bt.feeds.PandasData(
        dataname=testdata1,
        datetime='datetime',
        open='open',
        high='high',
        low='low',
        close='close',
        volume='volume',
        openinterest=None,
)

test_datafeeds = [test_datafeed0, test_datafeed1]

In [ ]:
best_params['verbose']=2
best_params['prefix']= 'Test'
# test_res = cerebro_run(test_datafeeds, best_params)

test_res = cerebro_run(test_datafeeds, best_params)

# Analyze with [Pyfolio](https://pyfolio.ml4trading.io/api-reference.html)

#TODO: We need find a way to calculate the per arbitrage data!... or should we?

Default package has an known error issue out of lack of maintainence, requires to modify the package manually
> https://github.com/quantopian/pyfolio/issues/652

Created a github merge
> https://github.com/quantopian/pyfolio/pull/703

### Analyze the grid search result

In [ ]:
grid_tradeanalyzer = best_result[0].analyzers.tradeanalyzer.get_analysis()
res_analyse(grid_tradeanalyzer)

grid_pyfolio = pyfolio_process(best_result)
pyfolio.tears.create_full_tear_sheet(returns=pd.Series(grid_pyfolio))

### Analyze the test result

In [ ]:
res_tradeanalyzer = test_res[0].analyzers.tradeanalyzer.get_analysis()
res_analyse(res_tradeanalyzer)

test_pyfolio = pyfolio_process(test_res)
pyfolio.tears.create_full_tear_sheet(returns=pd.Series(test_pyfolio))